In [31]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import datetime

In [2]:
# Initialize Data Directory

# Symbol
# Select folders by DOE > Today
# Get the latest data

## Plotting Arrangements

### General Structure

#### By Expiration ATM
- Option Type (P / C / P & C)
- Exact DOE / TTE Period (1M / 3M / 6M / 1Y / 1Y6M / 2Y / Max) / Full Normal Cycles (No weeklys) / Weeklys
- Metric: Nominal Price / OI / Vol / IV / delta / gamma / vega / theta / rho

#### By Strike Price (EXP)
- Option Type (P / C / P & C)
- Strike Range (numerical range, full)
- Exact DOE / TTE Period (1M / 3M / 6M / 1Y / 1Y6M / 2Y / Max) / Full Normal Cycles (No weeklys) / Weeklys
- Metric: Nominal Price / OI / Vol / IV / delta / gamma / vega / theta / rho


In [22]:
# Retrieve the Expiration arrays (X - axis)

# Process the files
# Locate the ticker folder


ticker = 'SPY'

# indir = '../Data/' + ticker + '/' + str(each) + '/'
        # if not os.path.exists(outdir):
            # os.makedirs(outdir)

# Traverse the expirations (past today -> locate latest file)

'04'

In [45]:
exprations = os.listdir(path='../Data/' + ticker)

# Filter dates past today:
# Check year - this year >= 0
# Check month - this month >= 0
# Check day - this day >= 0

valid_expirations = []
today_y = datetime.datetime.today().strftime('%y')
today_m = datetime.datetime.today().strftime('%m')
today_d = datetime.datetime.today().strftime('%d')

for each in exprations:

    # Add current year and months beyond current
    if int(pd.to_datetime(each).strftime('%y')) == int(today_y) and int(pd.to_datetime(each).strftime('%m')) > int(today_m):
        valid_expirations.append(each)
    # Add the following years
    if each not in valid_expirations and int(pd.to_datetime(each).strftime('%y')) > int(today_y):
        valid_expirations.append(each)
    # Add remaining expirations in the current month past today
    if each not in valid_expirations and int(pd.to_datetime(each).strftime('%y')) == int(today_y) and int(pd.to_datetime(each).strftime('%m')) == int(today_m) and int(pd.to_datetime(each).strftime('%d')) >= int(today_d):
        valid_expirations.append(each)

In [46]:
print(valid_expirations)

['2022-6-30', '2023-3-31', '2022-5-20', '2023-12-15', '2022-4-18', '2022-4-27', '2022-4-20', '2022-4-29', '2022-4-11', '2022-5-6', '2022-12-30', '2022-5-9', '2023-1-20', '2024-1-19', '2022-7-15', '2022-9-30', '2022-4-4', '2022-5-13', '2022-9-16', '2022-5-2', '2022-8-19', '2022-5-4', '2022-4-14', '2022-4-13', '2022-4-25', '2022-4-22', '2022-12-16', '2023-6-16', '2022-6-17', '2024-12-20', '2022-4-6', '2022-4-8', '2023-3-17']


In [ ]:
def plot_expiration_atm():

In [ ]:
def plot_strike_expiration():
    

In [6]:
# Plots (Takes)

In [3]:
# 